In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
from bs4 import BeautifulSoup


import pandas as pd
import numpy as np
import requests
import time
import re

In [29]:
df = pd.read_csv('kakaoAPI_data.csv', encoding='utf-8')

In [30]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
df.head()

,ID,stores,X,Y,road_address,place_url
0,20908598,공차 영등포구청점,126.897061,37.526699,서울 영등포구 당산로 136,http://place.map.kakao.com/20908598
1,1714644708,이도숙성회,126.896237,37.526695,서울 영등포구 당산로31길 4,http://place.map.kakao.com/1714644708
2,27527233,까치네분식,126.896689,37.527085,서울 영등포구 당산로 139,http://place.map.kakao.com/27527233
3,22246273,푸른농장 해바라기생고기정육식당 영등포점,126.896155,37.527045,서울 영등포구 당산로33길 5,http://place.map.kakao.com/22246273
4,19609040,사조참치 영등포구청점,126.896554,37.526706,서울 영등포구 당산로 133,http://place.map.kakao.com/19609040


In [31]:
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

/var/folders/6b/s1063_qj51s2xh9qm5l7ntq40000gn/T/ipykernel_1972/1284927200.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [32]:
# 수집할 정보
menu = []
category = []
business_hours = []
break_time = [] 
rating = []
review_count = []
phone_number = [] 
last_update = []
delay = 2

p = re.compile('[a-zA-Z]+')

In [33]:
for url in tqdm(df['place_url']):
    driver.get(url)
    time.sleep(delay)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    data = soup.find_all('div', 'mArticle')
    for x in data:
        category.append(x.find('span', 'txt_location').text)
        business_hours.append(x.find('span', 'txt_operation').text)
        break_time.append(x.find('span', 'txt_operation').text)
        rating.append(x.find('span', 'color_b').text)
        phone_number.append(x.find('span', 'txt_contact').text)
        last_update.append(x.find('span', 'date_revise').text)
        menues = soup.find_all('div', 'info_menu')
    
        menu_temp = []
        for y in menues:
            menu_temp.append(y.find('span', 'loss_word').text)
        menu.append(menu_temp)
        
        review_temp = x.find_all('span', 'color_g')
        if p.match(review_temp[0]) == None:
            review_count.append(review_temp[0].text)
        else:
            review_count.append(review_temp[1].text)
    time.sleep(delay)

100%|██████████| 1412/1412 [1:41:53<00:00,  4.33s/it]


In [34]:
df['menu'] = menu
df['category'] = category
df['business_hours'] = business_hours 
df['rating'] = rating
df['review_count'] = review_count
df['phone_number'] = phone_number
df['last_update'] = last_update
df['break_time'] = break_time
df.to_csv('kakaomapAPI_crawling(selenium).csv', encoding='utf-8-sig', index=False)

ValueError: Length of values (0) does not match length of index (1412)

In [35]:
print(len(menu))

0
